<a href="https://colab.research.google.com/github/sskyau/rl-super-mario/blob/main/rl_super_mario_bros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install nes-py==0.2.6
!pip install nes-py
!pip install gym-super-mario-bros
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y
!apt install -y libgl1-mesa-glx
!pip install opencv-python
# !pip install cupy

     |████████████████████████████████| 76 kB 2.8 MB/s 
  Created wheel for nes-py: filename=nes_py-8.1.8-cp37-cp37m-linux_x86_64.whl size=439813 sha256=5cec1ae3606cd1f9fd38847ea266a175253303c1825cd2d669be569af9f0843f
  Stored in directory: /root/.cache/pip/wheels/f2/05/1f/608f15ab43187096eb5f3087506419c2d9772e97000f3ba025
Successfully built nes-py
     |████████████████████████████████| 198 kB 5.1 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x8

In [ ]:
import torch
import torch.nn as nn
import random
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from tqdm import tqdm
import pickle 
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
import numpy as np
# import cupy as np
import collections 
import cv2
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
from gym import wrappers
from gym.wrappers.frame_stack import FrameStack
from gym.wrappers.gray_scale_observation import GrayScaleObservation
from gym.wrappers.resize_observation import ResizeObservation
from gym.wrappers import AtariPreprocessing

In [ ]:
# class MaxAndSkipEnv(gym.Wrapper):
#     def __init__(self, env=None, skip=4):
#         """Return only every `skip`-th frame"""
#         super(MaxAndSkipEnv, self).__init__(env)
#         # most recent raw observations (for max pooling across time steps)
#         self._obs_buffer = collections.deque(maxlen=2)
#         self._skip = skip

#     def step(self, action):
#         total_reward = 0.0
#         done = None
#         for _ in range(self._skip):
#             obs, reward, done, info = self.env.step(action)
#             self._obs_buffer.append(obs)
#             total_reward += reward
#             if done:
#                 break
#         max_frame = np.max(np.stack(self._obs_buffer), axis=0)
#         return max_frame, total_reward, done, info

#     def reset(self):
#         """Clear past frame buffer and init to first obs"""
#         self._obs_buffer.clear()
#         obs = self.env.reset()
#         self._obs_buffer.append(obs)
#         return obs


# class ProcessFrame84(gym.ObservationWrapper):
#     """
#     Downsamples image to 84x84
#     Greyscales image

#     Returns numpy array
#     """
#     def __init__(self, env=None):
#         super(ProcessFrame84, self).__init__(env)
#         self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

#     def observation(self, obs):
#         return ProcessFrame84.process(obs)

#     def process(frame):
#         if frame.size == 240 * 256 * 3:
#             img = np.reshape(frame, [240, 256, 3]).astype(np.float32)
#         else:
#             assert False, "Unknown resolution."
#         img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
#         resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
#         x_t = resized_screen[18:102, :]
#         x_t = np.reshape(x_t, [84, 84, 1])
#         return x_t.astype(np.uint8)


# class ImageToPyTorch(gym.ObservationWrapper):
#     def __init__(self, env):
#         super(ImageToPyTorch, self).__init__(env)
#         old_shape = self.observation_space.shape
#         self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]),
#                                                 dtype=np.float32)

#     def observation(self, observation):
#         return np.moveaxis(observation, 2, 0)


# class ScaledFloatFrame(gym.ObservationWrapper):
#     """Normalize pixel values in frame --> 0 to 1"""
#     def observation(self, obs):
#         return np.array(obs).astype(np.float32) / 255.0


# class BufferWrapper(gym.ObservationWrapper):
#     def __init__(self, env, n_steps, dtype=np.float32):
#         super(BufferWrapper, self).__init__(env)
#         self.dtype = dtype
#         old_space = env.observation_space
#         self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
#                                                 old_space.high.repeat(n_steps, axis=0), dtype=dtype)

#     def reset(self):
#         self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
#         return self.observation(self.env.reset())

#     def observation(self, observation):
#         self.buffer[:-1] = self.buffer[1:]
#         self.buffer[-1] = observation
#         return self.buffer


# def make_env(env, frame_stack):
#     env = MaxAndSkipEnv(env)
#     env = ProcessFrame84(env)
#     env = ImageToPyTorch(env)
#     env = BufferWrapper(env, 4)
#     env = ScaledFloatFrame(env)
#     return JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Custom class to make end of life = end of episode
# class SkipFrame(gym.Wrapper):

#   def __init__(self, env, n):
#     super(SkipFrame, self).__init__(env)
#     self._n = n
  
#   def step(self, action):
#     sum_rewards = 0
    
#     # Re-run every step in the n-frames and accumulate the info to return
#     done = False
#     for i in range(self._n):
#       #print('before step: ', done)
#       state, reward, done, info = self.env.step(action)
#       #print('after step: ', done)
#       sum_rewards += reward
#       if done:
#         break
  
    # return state, sum_rewards, done, info

# from atari_wrappers: https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

# from atari_wrappers: https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py
class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped._life
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped._life
        return obs

class CustomReward(gym.Wrapper):
  def __init__(self, env):
    super(CustomReward, self).__init__(env)
    self.score = 0
    self.y_pos = 0
    self.coins = 0

  def step(self, action):
    state, reward, done, info = self.env.step(action)
    # original reward function: (new_x_pos - old_x_pos) + (new_time - old_time) + dealth_pealty(-15 if dead; 0 otherwise)
    # clipped at (-15,15)
    
    # encourage Mario to jump to higher platforms (+5 reward if Mario stayes in y_pos >= 6 for consecutive frames)
    if (info['y_pos'] >= 6 and self.y_pos >= 6):
      reward += 5

    # reward Mario for killing monsters (+reward = (change in coins between this frame and last frame/100))
    reward += (info['coins'] - self.coins) / 100

    # info: {'coins': (int), 
    #        'flag_get': (bool), 
    #        'life': (int), 
    #        'stage': (int), 
    #        'status': (str) {'small', 'tall', 'firecall'}, 
    #        'time': (int) time left, 
    #        'world': (int) {1,...,8}, 
    #        'x_pos': (int), 
    #        'y_pos': (int)}

    # reward Mario for getting an upgrade
    if info['status'] != 'small':
      reward += 5

    if info['flag_get']:
      reward = 15
    self.y_pos = info['y_pos']
    self.coins = info['coins']
    
    return state, reward, done, info

class ScaleReshapeEnv(gym.ObservationWrapper):
    def __init__(self, env):
        super(ScaleReshapeEnv, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, 
                                                high=255.0, 
                                                shape=(old_shape[1], old_shape[2], old_shape[0]),
                                                dtype=np.float32)

    def observation(self, observation):
        observation = np.array(observation).astype(np.float32) / 255.0
        # print('before conversion ', observation.shape)
        shape = self.observation_space.shape
        observation = np.reshape(observation, (shape))
        return observation


def make_env(env, frame_size, frame_skip, frame_stack):
  env = JoypadSpace(env, SIMPLE_MOVEMENT)
  env = EpisodicLifeEnv(env)
  env = CustomReward(env) # customise reward
  # env = SkipFrame(env, frame_skip) 
  env = MaxAndSkipEnv(env, frame_skip) 
  env = GrayScaleObservation(env)
  env = ResizeObservation(env, frame_size) # downsample to (84,84) like DeepMind
  env = FrameStack(env, frame_stack)
  env = ScaleReshapeEnv(env) # normalise and reshape (e.g. (4,84,84) --> (84,84,4))
  return env

In [ ]:
class UniformExperienceReplay(object):
  def __init__(self, replay_size, mem_size):
    self.sample_size = replay_size
    self.mem_size = mem_size
    self.buffer = [None] * self.mem_size
    self.index = 0
  
  def push(self, s, a, r, d, s2):
    self.buffer[self.index] = s, a, r, d, s2
    self.index += 1

    # reset if memory is full
    if self.index == self.mem_size:
      self.index = 0
      self.buffer = [None] * self.mem_size
  
  def sample_exp(self):
    s = [None] * self.sample_size
    a = [None] * self.sample_size
    r = [None] * self.sample_size
    d = [None] * self.sample_size
    s2 = [None] * self.sample_size

    for batch, sample in enumerate(np.random.randint(0, self.index-1, self.sample_size)):
      s_sample, a_sample, r_sample, d_sample, s2_sample = self.buffer[sample]
      s[batch] = np.array(s_sample)
      a[batch] = a_sample
      r[batch] = r_sample
      d[batch] = d_sample
      s2[batch] = np.array(s2_sample)

    return np.array(s), np.array(a, dtype=int), np.array(r, dtype=float), np.array(d, dtype=bool), np.array(s2)

In [ ]:
from os import X_OK
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Lambda, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import backend as K

class DDQNMario:

    def __init__(self, state_space, action_space, mem_size, frame_stack, replay_size, gamma, lr, max_epsilon, min_epsilon, epsilon_decay, sync_rate, pretrained, burnin, load_model_epi):
      
      self.pretrained = pretrained 

      # Learning variables
      self.gamma = gamma
      self.epsilon_decay = epsilon_decay
      self.min_epsilon = min_epsilon
      self.epsilon = max_epsilon
      self.huber = keras.losses.Huber()
      self.burnin = burnin

      # Env variables
      self.step = 0
      self.state_space = state_space
      self.num_actions = action_space
      self.frame_stack = frame_stack

      # Experience replay variables
      self.mem_size = mem_size
      self.sync_rate = sync_rate
      self.buffer = 0
      self.replay_size = replay_size
      self.replayer = UniformExperienceReplay(replay_size, self.mem_size)
      # self.action_mask = np.ones(self.num_actions, dtype=int)


      # CNN network variables
      self.lr = lr
      self.optimiser = Adam(learning_rate=self.lr, clipnorm=1.0)
      self.online_hist = []

      # Init CNN networks
      if self.pretrained:
        # (Testing) use pre-trained weights for the models
        self.online = load_model('online_' + load_model_epi)
        self.target = load_model('target_' + load_model_epi)

      else:
        # (Training) inti models
        self.online = self.create_model()
        self.target = self.create_model()

      # Logging
      self.last_position = 0
      self.epi_exploitation = 0

    def create_model(self):
      inputs = Input(shape=(self.state_space))
      x = Conv2D(32, 3, strides=4, activation='relu', name='conv1')(inputs)
      x = Conv2D(64, 3, strides=2,  activation='relu', name='conv2')(x)
      x = Conv2D(64, 3, strides=1, activation='relu', name='conv3')(x)
      x = Flatten()(x)
      x = Dense(512, activation='relu')(x)
      action = Dense(self.num_actions, activation='softmax', name='action')(x) 

      model = tf.keras.Model(inputs=inputs, outputs=action)

      model.compile(optimizer=self.optimiser, loss=self.huber)
      #model.summary()
      return model

    def choose_action(self, state):
      self.step += 1

      # Epsilon-greedy policy #
      # Exploration 
      if random.random() < self.epsilon:  
        return random.randrange(self.num_actions)
      
      # Exploitation
      else:
        state = state[np.newaxis, :, :, :] # (84,84,4) --> (1,84,84,4)
        state = tf.convert_to_tensor(state, dtype=tf.float32)
        actions = self.online.predict(state)
        # print(np.argmax(actions))
        self.epi_exploitation += 1

        return np.argmax(actions[0])

    def sync_models(self):
      self.target.set_weights(self.online.get_weights())

    def experience_replay(self, ep_num):
      # reference: https://colab.research.google.com/github/ehennis/ReinforcementLearning/blob/master/06-DDQN.ipynb#scrollTo=4oVZuEP5vVTP
      if self.step % self.sync_rate == 0:
        self.sync_models()

      if self.replayer.index < self.replayer.sample_size:
        return 

      s = []
      y = [] 

      # sample minibatch of state, action, reward, done and state2 
      state_sample, action_sample, reward_sample, done_sample, state2_sample = self.replayer.sample_exp()

      state_sample = tf.convert_to_tensor(state_sample)
      state2_sample = tf.convert_to_tensor(state2_sample)

      # predict in batch
      state_pred_online = self.online.predict(state_sample) 
      state2_pred_online = self.online.predict(state2_sample)
      state2_pred_target = self.target.predict(state2_sample)

      for i in range(len(action_sample)):
        s.append(state_sample[i])
        
        state2_action_pred_online = state2_pred_online[i]
        state2_action_pred_target = state2_pred_target[i]
      
        if done_sample[i]:
          target = reward_sample[i] # y_j = r_j + 0 if terminal
        else:
          target = reward_sample[i] + self.gamma * state2_action_pred_target[np.argmax(state2_action_pred_online)] 
        
        # y_j
        target_f = state_pred_online[i] # prediction result (array of size 7)
        target_f[action_sample[i]] = target # 
        y.append(target_f)

      # s = np.array(s)
      # y = np.array(y)

      s = tf.convert_to_tensor(s, dtype=tf.float32)
      y = tf.convert_to_tensor(y, dtype=tf.float32)
      
      # perform single gradient update based on y_j
      self.online_hist = self.online.train_on_batch(s, y)

      # Update epsilon 
      self.epsilon = max(self.min_epsilon, self.epsilon*self.epsilon_decay)

    def log(self, state, action, reward, done, state2):
      # print('shape of state logged: ', state.shape)
      self.replayer.push(state, action, reward, done, state2)

In [ ]:
def show_state(env, ep=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("Episode: %d %s" % (ep, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
frame_stack = 4
def run(training_mode, pretrained, stage, model_epi=None):
   


    env = gym_super_mario_bros.make(stage) # world 1, stage 1, standard ROM
    env = make_env(env, 
                   frame_size=84,
                   frame_skip=4,
                   frame_stack=frame_stack)  # Wraps the environment so that frames are grayscale 
    state_space = env.observation_space.shape
    # print('observation space: ', env.observation_space.shape)
    action_space = env.action_space.n

    # Init
    num_episodes = 300
    burnin = int(num_episodes * 0.05)
    env.reset()
    total_rewards = []
    exploitation_rate = []

    agent = DDQNMario(state_space=state_space,
                      action_space=action_space,
                      frame_stack=frame_stack,
                      mem_size=15000,
                      replay_size=64,
                      gamma=0.90,
                      lr=.00025,
                      max_epsilon=1.0,
                      min_epsilon=0.1,
                      epsilon_decay=0.9999,
                      sync_rate = 10000, 
                      burnin = burnin,
                      pretrained=pretrained, 
                      load_model_epi=model_epi)
    
    best_reward = 0
    
    for ep_num in tqdm(range(num_episodes)):
        state = env.reset()
        # print(state)
        state = state
        total_reward = 0
        steps = 0
        agent.epi_exploitation = 0

        while True:

            if not training_mode: 
                show_state(env, ep_num)
            # print('choose action')
            action = agent.choose_action(state)
            # print('state shape fed into choose action',state.shape)
            steps += 1
            
            state2, reward, done, info = env.step(int(action))
            # print(state2)
            # print('env shape', env.observation_space.shape)
            # print('shape of state returned in step:', state2.shape)
            total_reward += reward
            state2 = state2
            
            if training_mode:
                agent.log(state, action, reward, done, state2)
                agent.experience_replay(ep_num)
            
            state = state2

            if done:
                exploitation_rate.append(agent.epi_exploitation / steps)
                break
        
        total_rewards.append(total_reward)

        print("Total reward after episode {} is {}. Exploitation rate is {}%.".format(ep_num + 1, total_rewards[-1], exploitation_rate[ep_num]*100))
        num_episodes += 1      

        # save the best models so far (with rewards > 40000)
        if total_reward > best_reward:
          best_reward = total_reward
          if total_reward > 40000:
            agent.online.save("online_"+ str(ep_num+1))
            agent.target.save("target_" + str(ep_num+1))

    if training_mode:
        # save training record 
        with open("ending_position.pkl", "wb") as f:
            pickle.dump(agent.last_position, f, protocol=4)
        with open("buffer.pkl", "wb") as f:
            pickle.dump(agent.replayer.buffer, f, protocol=4)
        
        agent.online.save("online")
        agent.target.save("target")

    env.close()
    
    if num_episodes > burnin:
        plt.title("Episodes trained vs. Average Rewards")
        plt.plot([0 for _ in range(burnin)] + 
            np.convolve(total_rewards, np.ones((burnin,))/burnin, mode="valid").tolist())
        plt.show()

In [ ]:
# Train model
run(training_mode=True, pretrained=False, stage='SuperMarioBros-1-1-v0')

  0%|          | 1/300 [01:04<5:22:51, 64.79s/it]

Total reward after episode 1 is 7848.02. Exploitation rate is 2.8089887640449436%.


  1%|          | 2/300 [02:58<7:46:05, 93.84s/it]

Total reward after episode 2 is 11181.98. Exploitation rate is 5.2734375%.


  1%|          | 3/300 [03:38<5:41:26, 68.98s/it]

...

Total reward after episode 110 is 6006.0. Exploitation rate is 87.45762711864407%.


 37%|███▋      | 111/300 [6:29:02<8:19:41, 158.63s/it]

Total reward after episode 111 is 9294.0. Exploitation rate is 91.8103448275862%.


 37%|███▋      | 112/300 [6:29:39<6:22:42, 122.14s/it]

Total reward after episode 112 is 2867.0. Exploitation rate is 93.5251798561151%.


In [ ]:
# Test model 
run(training_mode=False, pretrained=True, stage='SuperMarioBros-1-1-v0', model_epi='32')